# Firefly Visualization Demo

This notebook is intended to demonstrate the [Firefly](https://mospace.umsystem.edu/xmlui/handle/10355/5346) interactive interface for viewing image data. It also builds on the pedagogical explanations provided in [Getting started tutorial part 3](https://pipelines.lsst.io/getting-started/display.html) of the LSST Stack v16.0 documentation.

This tutorial seeks to teach you about how to use the LSST Science Pipelines to inspect outputs from single frame processing by displaying images and source catalogs in the Firefly image viewer. In doing so, you’ll be introduced to some of the LSST Science Pipelines’ Python APIs, including:

* Accessing datasets with the `Butler`.
* Displaying images with `lsst.afw.display`
* Pass source catalog data directly to the `FireflyClient`

## Set up

This tutorial is meant to be run from the `jupyterhub` interface where the LSST stack is preinstalled. It assumes that the notebook is running a kernel with the `lsst_distrib` package set up.

We start by importing packages from the LSST stack for data access and visualization.

In [ ]:
# LSST stack imports
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay

## Creating a Butler client

All data in the LSST Pipelines flow through the `Butler`. LSST does not recommend directly accessing processed image files. Instead, use the `Butler` client available from the `lsst.daf.butler` module imported above.

In [ ]:
datadir = '/project/shared/data/RSP_CHECK_REPO/butler.yaml'
butler = Butler(datadir)

The `Butler` client reads from the data repository specified with the input data path.  This is a rerun of some HSC data that is stored for posterity.

## Listing available data IDs in the Butler

To get data from the `Butler` you need to know three things: the dataset type, the data ID, and the collection to query.
This repository has been set up with a convenience collection, `HSC/defaults`, that packages up most collections a user may want to query.

Every dataset stored by the Butler has a well-defined type.
Tasks read specific dataset types and output other specific dataset types. 
Single frame processing reads in `raw` datasets and outputs `calexp`, or calibrated exposure, datasets (among others).
It’s `calexp` datasets that you’ll display in this tutorial.

Data IDs let you reference specific instances of a dataset.
On the command line you select data IDs with the `--d` argument, filtering by keys like `visit`, `detector`, `band`, and `physical_filter`.
The distinction between `band` and `physical_filter` is that the former is a label for any filter in the region of the electromagnetic spectrum e.g. 'i', and the latter is the specific name for the filter for a particular instrument 'HSC-I'.

Now, use the `Butler` client to find what data IDs are available for the `calexp` dataset type:



In [ ]:
results = butler.registry.queryDataIds(['visit', 'detector', 'band', 'physical_filter'], instrument='HSC', datasets='calexp', collections='HSC/defaults')
dataIds = []
for r in results:
    dataIds.append(r)
    print(r.full)

The printed output is a list of data IDs for the dataset type `calexp` in the repository. 

## Get an exposure through the Butler

Let’s get the dataset with the `Butler` client’s get method.  Note that the previous cell just tells you what data were ingested not what data were processed.  By inspection, the following data id was both ingested and processed.

In [ ]:
# There are two available calexp images in the repository.  The first has a very bright star in it, so we'll start visualization with the second
calexp = butler.get('calexp', dataId=dataIds[1], collections='HSC/defaults')

The `calexp` is an `ExposureF` Python object. Exposures are powerful representations of image data because they contain not only the image data, but also a variance image for uncertainty propagation, a bit mask image plane, and key-value metadata. They can also contain WCS and PSF model information. In the next steps you’ll learn how to display an Exposure’s image and mask.

## Create a Display

To display the `calexp` you will use the LSST `afwDisplay` framework. It provides a uniform API for multiple display backends, including DS9, matplotlib, and LSST’s Firefly viewer. The default backend is `ds9`, but since we are working remotely on `JupyterLab` we would prefer to use the web-based Firefly display. A [user guide](https://pipelines.lsst.io/v/daily/modules/lsst.display.firefly/index.html)  for `lsst.display.firefly` is available on the [pipelines.lsst.io](https://pipelines.lsst.io/v/daily) site.

Now, we create a Firefly display.

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.Display(frame=1)

In the Science Platform Notebook aspect, a Firefly viewer tab appears. You may wish to drag it to the right side of the JupyterLab area to create two side by side panes, one with the notebook and one with the display.

## Display the calexp (calibrated exposure)

We can now build the display and use the `mtv` method to view the `calexp` with Firefly. First we display an image with mask planes and then overplot some sources.

In [ ]:
afw_display.mtv(calexp)

As soon as you execute the command a single calibrated HSC exposure for the `{'physical_filter': 'HSC-I', 'detector': 87, 'visit': 1228}` data ID should appear in the Firefly `JupyterLab` tab.

Notice that the image is overlaid with colorful regions. These are mask regions. Each color reflects a different mask bit that correspond to detections and different types of detector artifacts. You’ll learn how to interpret these colors later, but first you’ll likely want to adjust the image display.

## Improving the image display

The display framework gives you control over the image display to help bring out image details. For example, to make masked regions semi-transparent, so that underlying image features are visible, try:

In [ ]:
afw_display.setMaskTransparency(80)

The setMaskTransparency method’s argument can range from 0 (fully opaque) to 100 (fully transparent).

You can also control the colorbar scaling algorithm with the display’s scale method. Try an asinh stretch with explicit minimum (black) and maximum (white) values:

In [ ]:
afw_display.scale("asinh", -5, 20)

You can also use an automatic algorithm like `zscale` (or `minmax`) to select the white and black thresholds:

In [ ]:
afw_display.scale("asinh", "zscale")

## Interpreting displayed mask colors

The display framework renders each plane of the mask in a different color (plane being a different bit in the mask). To interpret these colors you can get a dictionary of mask planes from the `calexp` and query the display for the colors it rendered each mask plane with. For example:

In [ ]:
mask = calexp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, afw_display.getMaskPlaneColor(maskName)))

In the Firefly viewer tab, the overlays button ![overlays button](http://irsa.ipac.caltech.edu/onlinehelp/finderchart/img/layers.png) on the toolbar gives you very detailed control over the mask planes, such as turning individual planes on and off, changing the color and adjusting the transparency. Mask transparency and colors can also be set using `afw.display` commands, for individual planes or for all.

## Plotting sources on the display

The LSST processing pipeline also creates a table of the sources it used for PSF estimation as well as astrometric and photometric calibration. The dataset type of this table is `src`, which you can get from the Butler:

In [ ]:
src = butler.get('src', dataId=dataIds[1], instrument='HSC', collections='HSC/defaults')

The returned object, `src`, is a `lsst.afw.table.SourceTable` object. `SourceTables` are explored more elsewhere, but you can do some simple investigations using common python functions. For example, to check the length of the object:




In [ ]:
len(src)

You can view an HTML rendering of the `src` table by getting an `astropy.table.Table` version of it:

In [ ]:
src.asAstropy()

Now we’ll overplot sources from the `src` table onto the image display using the Display’s `dot` method for plotting markers. `Display.dot` plots markers individually, so you’ll need to iterate over rows in the `SourceTable`.  It is more efficient to send a batch of updates to the display, so we enclose the loop in a `display.Buffering` context, like this:

In [ ]:
with afw_display.Buffering():
    for record in src:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

## Clearing markers

`Display.dot` always adds new markers to the display. To clear the display of all markers, use the erase method:

In [ ]:
afw_display.erase()

## Using FireflyClient directly.

We can also use the Firefly client directly to make plots and add catalogs to the visualization.  First retrieve the `FireflyClient` object.

In [ ]:
fc = afw_display.getClient()

For uploading a table it is convenient to use the firefly_client.plot module. Import it and ensure it is using the same FireflyClient instance.

In [ ]:
import firefly_client.plot as ffplt
ffplt.use_client(fc)

Let's select just the sources that were used in fitting the PSF. 

In [ ]:
psf_src = src[src['calib_psf_used']]
print(src['calib_psf_used'])

Upload a SourceCatalog to Firefly. By default, the catalog is shown in an interactive table viewer.

In [ ]:
tbl_id = ffplt.upload_table(psf_src, title='Source Catalog')

Make a scatter plot.

In [ ]:
ffplt.scatter(x_col='base_CircularApertureFlux_12_0_instFlux/base_GaussianFlux_instFlux',
              y_col='log10(base_CircularApertureFlux_12_0_instFlux)',
              size=4,
              color='blue',
              title='test ap flux/model mag vs. log(ap flux)',
              xlabel='Model',
              ylabel='Ap/Model')

Plots are rendered by `plotly`, so follow the same syntax for construction.  For info on `plotly` see the [primer](https://plot.ly/python/getting-started/) and [examples](https://plot.ly/python/line-and-scatter/).

## Wrap up
In this tutorial we have used the `Butler` to access LSST simulation data and used the LSST Science Pipelines Python API to display images and tables. Here are some key takeaways:

* Use the `lsst.daf.persistence.Butler` class to read and write data from repositories.
* The `lsst.afw.display` module provides a flexible framework for sending data from LSST Science Pipelines code to image displays. We used the Firefly backend for web-based visualization (`ds9`, `matplotlib`, and `ginga` are other avialable backends).
* `Exposure` objects have image data, mask data, and metadata. When you display an Exposure, the display framework automatically overlays mask planes.
* We have accessed and visualized the Table of sources extracted from an image. The `Table.asAstropy` method can be used to view the table as an `astropy.table.Table`.
* We have sent a subset of objects to directly to the Firefly API for additional plotting and investigation.


